In [1]:
%load_ext autoreload
%autoreload 2

# LLaMaBot's PydanticBot in under 5 minutes

We want to be able to pull structured data out of unstructured text. When the data is structured, we can the use programmatically in later steps.

In this example, we'll look at a small dataset of SciPy videos uploaded to YouTube. The videos are given a title and a description. We want to extract the name of the speaker giving the talk, and the topics the talk is about.
We want to be able to validate the data we've extracted not only matches the structured format we expect, but that it also meets some custom requirements.

In [2]:
# load in unstructured text data
import pandas as pd

df = pd.read_json("../scipy_videos.json", orient="index")
df

,name,description,view_count
0ALKGR0I5MA,Basic Sound Processing in Python | SciPy 2015 ...,,261832
ZB7BZMhfPgk,Introduction to Numerical Computing with NumPy...,NumPy provides Python with a powerful array pr...,208823
v5ijNXvlC5A,Modern Time Series Analysis | SciPy 2019 Tutor...,This tutorial will cover the newest and most s...,199372
tYYVSEHq-io,Getting Started with TensorFlow and Deep Learn...,"A friendly introduction to Deep Learning, taug...",161483
xAoljeRJ3lU,A Better Default Colormap for Matplotlib | Sci...,Complete SciPy 2015 Talk & Tutorial Playlist h...,160912
5rNu16O3YNE,Introduction to Data Processing in Python with...,This is a tutorial for beginners on using the ...,117260
JNfxr4BQrLk,Time Series Analysis with Python Intermediate ...,Tutorial materials for the Time Series Analysi...,113018
KhAUfqhLakw,Frequentism and Bayesianism: What's the Big De...,,106990
gtejJ3RCddE,NumPy Beginner | SciPy 2016 Tutorial | Alexand...,Materials for this tutorial may be found here:...,106043
nq6iPZVUxZU,UMAP Uniform Manifold Approximation and Projec...,This talk will present a new approach to dimen...,96781


Using Pydantic, we can define a class and some validation rules

In [3]:
from typing import List, Optional
from pydantic import BaseModel, Field, field_validator

class TopicExtract(BaseModel):
    """ This object extracts the name of the speaker presenting the video, and a list of topics that best describe what this talk is about."""
    
    speaker_name: Optional[str] = Field(default=None, description="The name of the speaker giving this talk. If there is no speaker named, leave empty.")
    topics: List[str] = Field(
        description="A list of upto 5 topics that this text is about. Each topic should be at most 1 or 2 word descriptions.")

    @field_validator('speaker_name')
    def validate_name(cls, speaker_name):
        return speaker_name
    
    @field_validator('topics')
    def validate_topics(cls, topics):
        # validate that the list of topics contains atleast 1, and no more than 5 topics 
        if len(topics) <= 0 or len(topics) > 5:
            raise ValueError('The list of topics can be no more than 5 items')

        # for each topic the model generated, ensure that the topic contains no more than 2 words
        for topic in topics:
            if len(topic.split()) > 2:
                # make the validation message helpful to the LLM.
                # Here we repeat which topic is failing validation, and remind it what it must do to pass the validation.
                raise ValueError(f'The topic "{topic}" has too many words, A topic can contain AT MOST 2 words')
        return topics

Now we can initialize the PydanticBot and assign this model to it.

In [4]:
from llamabot.prompt_manager import prompt
from llamabot.bot.pydanticbot import PydanticBot


system_prompt = "You are an expert topic labeller. You read a video title and description and extract the speakers name and the topics the video is about."

bot = PydanticBot(
    system_prompt=system_prompt,
    session_name="session_name",
    model_name = "ollama/llama3:latest",
    temperature=0,
    stream_target="stdout",
    pydantic_model=TopicExtract
)

Now we can pass in our text, and extract the topics

In [5]:
video_extracts = []
for index, video_row in df.iterrows():
    video_text = f"video title: {video_row['name']}\nvideo description: {video_row['description']}"

    extract = bot(video_text)
    
    video_extracts.append(extract)

{"speaker_name": "Allen Downey", "topics": ["Basic Sound Processing", "Python", "SciPy", "Audio Processing"]}{"speaker_name": "Allen Downey", "topics": ["Python", "SciPy", "Audio Processing"]}{"speaker_name": "Alex Chabot-Leclerc", "topics": ["Numerical Computing", "NumPy", "SciPy", "Array Processing", "Python"]}
{"speaker_name": "Aileen Nielsen", "topics": ["Time Series Analysis", "Bayesian Methods", "Machine Learning", "Deep Learning", "Python"]}
{"speaker_name": "Aileen Nielsen", "topics": ["Time Series", "Bayesian Methods", "Machine Learning", "Deep Learning", "Python"]}{"speaker_name": "Josh Gordon", "topics": ["Deep Learning", "TensorFlow", "Machine Learning", "Computer Vision", "Natural Language Processing"]}{"speaker_name": "Josh Gordon", "topics": ["Deep Learning", "TensorFlow", "Machine Learning", "Computer Vision", "Natural Language Processing"]}{"speaker_name": "Josh Gordon", "topics": ["Deep Learning", "TensorFlow", "Machine Learning", "Computer Vision", "Natural Lang"]}{"

In [7]:
for video in video_extracts:
    print(video)

speaker_name='Allen Downey' topics=['Python', 'SciPy', 'Audio Processing']
speaker_name='Alex Chabot-Leclerc' topics=['Numerical Computing', 'NumPy', 'SciPy', 'Array Processing', 'Python']
speaker_name='Aileen Nielsen' topics=['Time Series', 'Bayesian Methods', 'Machine Learning', 'Deep Learning', 'Python']
speaker_name='Josh Gordon' topics=['Deep Learning', 'TensorFlow', 'Machine Learning', 'Computer Vision', 'Natural Lang']
speaker_name='Nathaniel Smith and Stéfan van der Walt' topics=['Matplotlib', 'Default Colormap', 'SciPy']
speaker_name='Daniel Chen' topics=['Data Processing', 'Python', 'Pandas', 'Data Manipulation', 'Data Analysis']
speaker_name='Aileen Nielsen' topics=['Time Series', 'Python', 'SciPy']
speaker_name='Jake VanderPlas' topics=['Frequentism', 'Bayesianism', 'Statistics', 'Machine Learning', 'SciPy']
speaker_name='Alexandre Chabot LeClerc' topics=['NumPy', 'SciPy', 'Python', 'Array Processing', 'Mathematical Functions']
speaker_name=None topics=['Dimension Reduction